### Домашняя работа по теме "Введение в задачу классификации. Постановка задачи и подготовка данных."

##### 1. Приведите по 2 примера, когда лучше максимизировать Precision, а когда Recall.

<u>Максимизация Precision:</u>

a) Модель-советник для торговли на фондовом рынке. Классифицирует данные по котировкам ценных бумаг.
Целевой класс - "покупать". Тут нам важна уверенность в принятом решении, чтобы не потерять деньги,
даже если некоторые потенциально прибыльные сигналы при этом мы можем пропустить.

б) Производственная линия на химическом заводе. По составу продукта модель определяет, соответствует ли он
характеристикам для продажи конечному потребителю, либо для перевода на следующий этап производства.
Целевой класс - "соответствует". Важна точность предсказания, т.к. существуют финансовые риски. Не так
страшно забраковать хороший продукт, как выкатить на продажу плохой.  
Кстати, если мы целевым классом выберем "не соответствует", то на первый план выходит Recall.
Поправьте, пожалуйста, если не прав.

<u>Максимизация Recall:</u>

а) Модель, которая классифицирует отзывы в Интернете на плохие/хорошие. Целевой класс - "плохой отзыв".
Здесь важно найти как можно больше плохих отзывов, несмотря на то, что в этот класс попадёт какое-то количество
хороших отзывов.

б) Радиотелескоп, который фиксирует сигналы различных параметров из космоса. По этим параметрам модель определяет,
является ли сигнал интересным для учёных или это просто "шум". Целевой класс - "сигнал интересен". Важно цепляться
за каждый потенциально интересный сигнал, даже если в результате дальнейшего изучения он таковым не окажется.

##### 2. Почему мы используем F-меру, почему, например, нельзя просто взять среднее от Precision и Recall?

In [1]:
from sklearn.metrics import f1_score, precision_score, recall_score

Возьмём пример из задачи № 1, где у нас торговый советник.  
Составим массив pred, содержащий сигналы советника, где 1 - сигнал на покупку, 0 - отсутствие сигнала.  
Также возьмём массив true, в котором 1 - нужно было покупать, 0 - ничего не надо было делать.  
F-меру будем рассматривать на примере F1_score ($\beta$ = 1), так как в данном случае это не принципиально.

In [2]:
true = [0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0]
pred = [0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0]

In [3]:
# Напишем функцию для исключения дублирования кода
def evaluate(true, pred):
    print('Precision:')
    print(f'{precision_score(true, pred)}\n')
    print('Recall:')
    print(f'{recall_score(true, pred)}\n')
    print('F1_score:')
    print(f'{f1_score(true, pred)}\n')
    print('Среднее между Precision и Recall:')
    print(f'{(precision_score(true, pred) + recall_score(true, pred)) / 2}')

Рассчитаем метрики этой модели:

In [4]:
evaluate(true, pred)

Precision:
0.8

Recall:
0.6666666666666666

F1_score:
0.7272727272727272

Среднее между Precision и Recall:
0.7333333333333334


Теперь представим, что наша модель во всех ситуациях советует нам покупать:

In [5]:
pred = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Теперь метрики выглядят следующим образом:

In [6]:
evaluate(true, pred)

Precision:
0.4

Recall:
1.0

F1_score:
0.5714285714285715

Среднее между Precision и Recall:
0.7


А теперь наоборот - пусть модель практически никогда не даёт нам сигналов:

In [7]:
pred = [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

In [8]:
evaluate(true, pred)

Precision:
1.0

Recall:
0.16666666666666666

F1_score:
0.2857142857142857

Среднее между Precision и Recall:
0.5833333333333334


Первоначально метрика f1_score и средняя между precision и recall были примерно равны.  
Когда мы изменили предсказания и модель начала всегда советовать нам покупать, средняя двух метрик упала незначительно, а F1_score упала сильнее - с 0.72 до 0.57, - указывая на то, что качество предсказаний модели существенно снизилось (так и есть, хоть мы и получили максимальный recall, точность оставляет желать лучшего).  
Когда модель дала всего одно предсказание, мы получили максимальную точность и очень маленький recall. При этом средняя двух метрик упала всего лишь до 0.58, а f1_score - до 0.28.  
  
<u>Вывод:</u> В сравнении со средней величиной между Precision и Recall, метрика f1_score существенно снижается, когда падает хотя бы что-то одно: точность или полнота. Это логично, учитывая, что в числителе этой метрики произведение двух метрик, тогда как среднее имеет дело только с суммой. То есть, среднему не важно, если у нас одна из метрик сильно упала, лишь бы другая компенсировала это падение. Но реальные модели должны быть хороши и в точности, и в полноте, и f1_score отслеживает эту комбинацию.

##### 3. *Реализовать функции для подсчета Accuracy, Precision, Recall, F-score, которые на вход принимают y_true (истинные значения), y_pred (предсказанные значения), а на выход дается метрика.

Так как по условию задачи функции должны принимать на вход только y_true и y_pred, то F-score будем считать с $\beta$ = 1

In [9]:
from sklearn.metrics import accuracy_score
import numpy as np

Зададим явно значения y_true и y_pred для возможности проверки работы функций:

In [10]:
y_true = np.array([0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0])
y_pred = np.array([0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0])

Создадим класс с функциями для подсчёта метрик и сравнения их со встроенными функциями подсчёта:

In [11]:
class MyMetrics:
    
    def __init__(self, y_true, y_pred):
        
        self.y_true = y_true
        self.y_pred = y_pred
        
        # True Negative
        self.TN = ((self.y_true == 0) & (self.y_pred == 0)).sum()
        
        # False Positive
        self.FP = ((self.y_true == 0) & (self.y_pred == 1)).sum()
        
        # False Negative
        self.FN = ((self.y_true == 1) & (self.y_pred == 0)).sum()
        
        # True Positive
        self.TP = ((self.y_true == 1) & (self.y_pred == 1)).sum()
    
    def my_accuracy(self):
        return (self.TP + self.TN) / (self.TP + self.FP + self.TN + self.FN)
    
    def my_precision(self):
        return self.TP / (self.TP + self.FP)
    
    def my_recall(self):
        return self.TP / (self.TP + self.FN)
    
    def my_fscore(self):
        return 2 * self.my_precision() * self.my_recall() / (self.my_precision() + self.my_recall())
    
    def compare(self):
        
        print(f'My accuracy score:\t\t{self.my_accuracy()}\nBuilt-in accuracy score:\t{accuracy_score(self.y_true, self.y_pred)}\n')
        
        print(f'My precision score:\t\t{self.my_precision()}\nBuilt-in precision score:\t{precision_score(self.y_true, self.y_pred)}\n')
        
        print(f'My recall score:\t{self.my_recall()}\nBuilt-in recall score:\t{recall_score(self.y_true, self.y_pred)}\n')
        
        print(f'My f1-score:\t\t{self.my_fscore()}\nBuilt-in f1-score:\t{f1_score(self.y_true, self.y_pred)}')

Создадим объект класса для подсчёта метрик:

In [12]:
model = MyMetrics(y_true, y_pred)

Сравним наши метрики со встроенными:

In [13]:
model.compare()

My accuracy score:		0.8
Built-in accuracy score:	0.8

My precision score:		0.7142857142857143
Built-in precision score:	0.7142857142857143

My recall score:	0.8333333333333334
Built-in recall score:	0.8333333333333334

My f1-score:		0.7692307692307692
Built-in f1-score:	0.7692307692307692
